In [19]:
import torch 
import torch.nn as nn 
import h5py

In [20]:
with h5py.File('/home/kitadam/ENR_Sven/moxie/data/processed/pedestal_profile_for_nesep.hdf5', 'r') as file:
    X, y = file['strohman']['X'][:], file['strohman']['y'][:]

In [21]:
X, y = torch.from_numpy(X), torch.from_numpy(y)
# A pseudo batch 
X, y = X[:512], y[:512]

In [22]:
X.shape

torch.Size([512, 63])

The dimensions need to be expanded, as we Conv layers expect a shape of `[batch_size, channels, length]`

For us, the channels is 1, for 1 type of time series (at the moment just density profiles)

In [23]:
X = X.unsqueeze(1)

In [24]:
X.shape

torch.Size([512, 1, 63])

Now lets make some convolution layers 

In [27]:
conv_1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1)
conv_2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1)
conv_3 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1)

We pass the through each conv layer, printing the size as we go

In [28]:
out_1 = conv_1(X)
print('First conv {}'.format(out_1.shape))
out_2 = conv_2(out_1)
print('Second conv {}'.format(out_2.shape))
out_3 = conv_3(out_2)
print('Third conv {}'.format(out_3.shape))

First conv torch.Size([512, 4, 61])
Second conv torch.Size([512, 8, 59])
Third conv torch.Size([512, 16, 57])


As you can see, the length is slowly decreasing! We can change this to be constant to the relative shape of it all. 

In [29]:
conv_1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1, padding='same')
conv_2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding='same')
conv_3 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding='same')

In [30]:
out_1 = conv_1(X)
print('First conv {}'.format(out_1.shape))
out_2 = conv_2(out_1)
print('Second conv {}'.format(out_2.shape))
out_3 = conv_3(out_2)
print('Third conv {}'.format(out_3.shape))

First conv torch.Size([512, 4, 63])
Second conv torch.Size([512, 8, 63])
Third conv torch.Size([512, 16, 63])


Now we want to be able to feed this to our latent space, which means we have to flatten it back to its original shape `[batch_size, new_length]`, where new length will be our last channel size times our input vector size, which is in our case `63*16=1008`. 

In [34]:
encoded = out_3.view(out_3.size(0),-1)
encoded.shape

torch.Size([512, 1008])

Now we represent the latent space, which has hidden dimension 10,  with the variables mu and logvar

In [36]:
fc_mu, fc_logvar = nn.Linear(1008, 10), nn.Linear(1008, 10)
mu, logvar = fc_mu(encoded), fc_logvar(encoded)
mu.shape, logvar.shape

(torch.Size([512, 10]), torch.Size([512, 10]))

Now we reparameterize the mu and logvar

In [38]:
std = torch.exp(0.5*logvar)
eps = torch.randn_like(std)
z = eps * std + mu
z.shape

torch.Size([512, 10])

Now we want to feed it back into the transposed convolution layers, so we need to go from our hidden dimension 10 back to 1008 `(16*63)` using a fully connected layer

In [45]:
decoder_input = nn.Linear(10, 1008)
decoder_ready = decoder_input(z)
decoder_ready.shape

torch.Size([512, 1008])

But we have to do the inverse of before so we have to unflatten it to feed back into the convolution steps

In [46]:
unflattened = decoder_ready.view(decoder_ready.size(0), 16, 63)
unflattened.shape

torch.Size([512, 16, 63])

Now we design our 3 deconvolution steps

In [52]:
t_conv3 = nn.ConvTranspose1d(in_channels=16, out_channels=8, kernel_size=3, stride=1, padding=1)
t_conv2 = nn.ConvTranspose1d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=1)
t_conv1 = nn.ConvTranspose1d(in_channels=4, out_channels=1, kernel_size=3, stride=1, padding=1)

In [54]:
d_out3 = t_conv3(unflattened)
print('First transposed conv {}'.format(d_out3.shape))
d_out2 = t_conv2(d_out3)
print('Second transposed conv {}'.format(d_out2.shape))
d_out1 = t_conv1(d_out2)
print('First transposed conv {}'.format(d_out1.shape))

First transposed conv torch.Size([512, 8, 63])
Second transposed conv torch.Size([512, 4, 63])
First transposed conv torch.Size([512, 1, 63])


We could run this through another few linear layers of arbitrary size, but the output is how we want it. 

In [56]:
final_layer = nn.Sequential(nn.Linear(63, 63), nn.Linear(63, 63))
final = final_layer(d_out1)
final.shape

torch.Size([512, 1, 63])